In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import requests
import math

device = torch.device('mps')
torch.manual_seed(0)

In [ ]:
res = requests.get('https://assets.datacamp.com/production/repositories/3937/datasets/213ca262bf6af12428d42842848464565f3d5504/sherlock.txt')
dat = str((res.content))
import re

def replace_newlines(text):
    pattern = r'\\n'
    replaced_text = re.sub(pattern, ' ', text)
    return replaced_text


text = replace_newlines(dat)
print(text[:500])

In [ ]:
from transformers import AutoTokenizer

tokz = AutoTokenizer.from_pretrained('openai-community/gpt2')

tokenized_text = tokz.encode(text,truncation=True,return_overflowing_tokens=True)


In [ ]:
n = int(0.9*len(tokenized_text)) # first 90% will be train, rest val
train_data = (tokenized_text[:n])
val_data = tokenized_text[n:]

In [ ]:
def get_batch(split, batch_size=32):
    # generate a batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    
    # get the total number of sequences in the data
    total_sequences = len(data)
    
    # generate random indices to select batches
    ix = torch.randint(total_sequences, (batch_size,))
    # select sequences from the data using the random indices
    
    x = torch.stack([torch.tensor(data[i]) for i in ix])
    y = torch.stack([torch.cat((seq[1:], seq[0].unsqueeze(0))) for seq in x])
    
    x, y = x.to(device), y.to(device)
    return x, y

x,y = get_batch('train')

In [ ]:
max_iters = 5000
eval_interval = 500
eval_iters = 200
learning_rate = 3e-4
device = 'mps'
n_embed = 384 # has to be divisible(without rem) by n_head, given head_size definition further below
n_head = 6
n_layer = 7
dropout = 0.3
vocab_size = len(tokz.get_vocab())
block_size = 1024
head_size = n_embed // n_head

# single head of self attention
class InfiniAttention(nn.Module):
    """ one head of self attention """

    def __init__(self):
        super().__init__()
        self.n_embed = n_embed
        self.num_heads = n_head
        self.head_dim = self.n_embed // self.num_heads
        self.attention_dropout = nn.Dropout(p = 0.1)
        
        self.q_proj = nn.Linear(self.n_embed,self.num_heads * self.head_dim,bias = True)
        self.k_proj = nn.Linear(self.n_embed,self.num_heads * self.head_dim,bias = True)
        self.v_proj = nn.Linear(self.n_embed,self.num_heads * self.head_dim,bias = True)
        self.o_proj = nn.Linear(self.num_heads * self.head_dim,self.n_embed,bias = False)

        # In Pytorch convention a variable that's not a parameter of the model is called a buffer
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.attention_drop_p = 0.1
        
        self.beta = nn.Parameter(torch.rand(1))
        self.register_buffer("M", torch.zeros(self.num_heads, self.head_dim, self.head_dim))
        self.register_buffer("z", torch.zeros(self.num_heads, self.head_dim))

    def forward(self, x:torch.tensor): # 32 x 1024 x 384 
        bs,q_len,_ = x.size()
        print(bs),print(q_len)
        q_states = self.q_proj(x) # 32 x 1024 x 384
        k_states = self.k_proj(x) # 32 x 1024 x 384
        v_states = self.v_proj(x) # 32 x 1024 x 384

        q_states = q_states.view(bs, q_len, self.num_heads, self.head_dim).transpose(1,2) # 32 x 6 x 1024 x 64
        k_states = k_states.view(bs, q_len, self.num_heads, self.head_dim).transpose(1,2) # 32 x 6 x 1024 x 64
        v_states = v_states.view(bs, q_len, self.num_heads, self.head_dim).transpose(1,2) # 32 x 6 x 1024 x 64
        
        kv_seq_len = k_states.shape[-2] # 1024
        attention_mask = torch.tril(torch.ones(q_len, kv_seq_len)).unsqueeze(0).repeat(bs, 1, 1, 1)
        attention_mask = attention_mask.to(x.device)
    
        if attention_mask.size() != (bs, 1, q_len, kv_seq_len):
            raise ValueError(
                f"Attention mask should be of size {(bs, 1, q_len, kv_seq_len)}, but is {attention_mask.size()}"
            )
        A_mem = self._retrieve(q_states, self.M, self.z)
        self.M, self.z = self._update(k_states, v_states, self.M, self.z) 

        A_dot = F.scaled_dot_product_attention(
            q_states,
            k_states,
            v_states,
            attn_mask = attention_mask,
            dropout_p = self.attention_drop_p
        )

        final_attention = self._attn_mul(A_mem = A_mem,A_dot = A_dot)
        
        final_attention = final_attention.transpose(1, 2).contiguous()
        final_attention = final_attention.view(bs, q_len, self.n_embed)
        out = self.o_proj(final_attention)

        return out

    def _retrieve(self, Q, M, z):
        # Retrieve from compressive memory eq.3
        M_s_1 = torch.matmul(F.elu(Q) + 1, M)
        z_s_1 = torch.matmul(F.elu(Q) + 1, z.unsqueeze(-1)) + 1e-8
        A_mem = M_s_1 / z_s_1
        return A_mem
    
    def _update(self, K, V, M, z, use_delta = False):
        eps = 1e-8
        if use_delta:
            v_retrieved = torch.matmul(F.elu(K) + 1, M) / torch.matmul(F.elu(K) + 1, z.unsqueeze(-1) + eps)
            updated_m = M + torch.matmul(F.elu(K).transpose(-1,-2) + 1, V - v_retrieved)
        else:
            updated_m = M  + torch.matmul(F.elu(K).transpose(-1,-2) + 1, V)

        updated_z = z + (F.elu(K) + 1).sum(dim = -2)
        M = updated_m
        z = updated_z

        return M, z
    
    def _attn_mul(self, A_dot, A_mem):
        beta = torch.sigmoid(self.beta)
        A = beta * A_mem + (1 - beta) * A_dot
        return A

In [ ]:
class FeedForward(nn.Module):
    " simple linear layer followed by non linearity "
    
    def __init__(self, n_embed):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embed, 4 * n_embed), # as mentioned in the paper
            nn.ReLU(),
            nn.Linear(4 * n_embed, n_embed), # projection layer : the final projection back into the residual pathway
            nn.Dropout(dropout), # dropout before final projection
        )
        
    def forward(self, x):
        return self.net(x)


class Block(nn.Module):
    """ a transformer block : communication then computation """
    
    def __init__(self, n_embed, n_head):
        super().__init__()
        head_size = n_embed // n_head
        self.sa = InfiniAttention()
        self.ffwd = FeedForward(n_embed)
        self.ln1 = nn.LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)
    
    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x


class LanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        self.position_embedding_table = nn.Embedding(block_size, n_embed)
        self.blocks = nn.Sequential(*[Block(n_embed, n_head=n_head) for _ in range(n_layer)]) 
        self.ln_f = nn.LayerNorm(n_embed)
        self.lm_head = nn.Linear(n_embed, vocab_size)
        
    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        token_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T, C)
        x = token_emb + pos_emb # (B, T, C)
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)# (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block size token
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = LanguageModel()
model.to(device)
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            X.to(device),Y.to(device)
            _ , loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

m = model.to(device)

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    # interesting that we're not printing loss every iter
    # instead we're estimating the non noisy loss every eval_intervar
    # only for printing purposes
    if (iter % eval_interval == 0) or (iter == max_iters-1):
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')
    xb.to(device),yb.to(device)
    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# # generate from the model
# context = torch.zeros((1, 1), dtype=torch.long, device=device)
# output = tokz.decode(m.generate(context, max_new_tokens=500)[0].tolist())